# Aspect Based Sentiment Analysis


### Data And Code

In [4]:
# import all necessary packeges
import pandas as pd
import xml.etree.ElementTree as ET, getopt, logging, sys, random, re, copy
from xml.sax.saxutils import escape
import io


from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")







## Fetching Data

In [5]:
# we have give a json file which consists of all information about
# the products
# loading the data using pandas' read_json file.
xml_data1 = open('D:\ABSA\semeval_data\Restaurants_Train.xml').read()

# Fetch the sentence data
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = [] #This is our record list which we will convert into a dataframe
    for i, child in enumerate(root): #Begin looping through our root tree
        record = {} #Place holder for our record
        for subchild in child: #iterate through the subchildren to user-agent, Ex: ID, String, Description.
            if(subchild.tag == 'text'):
                record[subchild.tag] = subchild.text
                all_records.append(record)
            #record[subchild.tag] = subchild.text #Extract the text create a new dictionary key, value pair
            #print(subchild.text)
            #all_records.append(record) #Append this record to all_records.
            
    return pd.DataFrame(all_records) #return records as DataFrame


def iter_docs(sentences):
    author_attr = sentences.attrib
    for doc in sentences.iter('sentence'):
        
        #print(doc.find('text'))
        doc_dict = author_attr.copy()
        #print(author_attr['aspectTerm'])
        #print(doc.attrib)
        doc_dict.update(doc.attrib)  # doc.attribute give the value of the sentence id e.g. {'id': 12334}
        #doc_dict['data'] = doc.text
        #print(doc_dict)
        yield doc_dict
        


xml_data = io.StringIO(xml_data1)

etree = ET.parse(xml_data) #create an ElementTree object 
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))


sentence_data = xml2df(xml_data1)



print(doc_df)
print(sentence_data)

        id
0     3121
1     2777
2     1634
3     2534
4      583
5     2846
6     1571
7     1458
8     3161
9     2391
10     986
11     296
12    1793
13    2200
14    3195
15    3119
16    3359
17      70
18    1829
19    2935
20    1805
21    2559
22     716
23    2464
24     385
25    1350
26    3534
27    1407
28     565
29     387
...    ...
3014   902
3015   568
3016  3133
3017  2553
3018  2624
3019  2116
3020   317
3021  2151
3022   944
3023  3014
3024   900
3025  2739
3026   279
3027  2937
3028   333
3029   236
3030  1420
3031  2259
3032   608
3033  2834
3034  3168
3035  2798
3036  2378
3037  1027
3038  1735
3039  1063
3040   777
3041   875
3042   671
3043   617

[3044 rows x 1 columns]
                                                   text
0                  But the staff was so horrible to us.
1     To be completely fair, the only redeeming fact...
2     The food is uniformly exceptional, with a very...
3     Where Gabriela personaly greets you and recomm...
4     For tho

In [102]:
print(sum(sentence_data.duplicated('text')))

6


#### For the Aspect Term Data

In [7]:
aspects_records = []
for node in etree.iter():
    record = {}
    
    #print(node.attrib)
    #print(node.tag)
    
    
    if(node.tag == 'sentence'):
        id = node.attrib.get('id')
        #print("1:",id)
        
    if(node.tag == 'aspectTerm' ):
        
        #print("2",id)
        record['id'] = id
        record['term'] = node.attrib.get('term')
        record['polarity'] = node.attrib.get('polarity')
        record['from'] = node.attrib.get('from')
        record['to'] = node.attrib.get('to')
        aspects_records.append(record)
    #print(node.attrib.get('id'))

    
    #record[node.tag] = node.attrib
    #all_records.append(record)
    
    #pd.DataFrame(all_records)
    #print ("Tag:",node.tag)
    
    #print("Sentence:",node.find('text').text)
    
    #print("Value:", node.attrib.get('id'))
    
    
    #print (node.attrib) record['aaa'] = '111'
   # print ("Term:",node.attrib.get('term'),"Polarity:", node.attrib.get('polarity'), "From:", node.attrib.get('From'), "To", node.attrib.get('to'))
   # print ("Polarity", node.attrib.get('polarity'))
   #print ("From", node.attrib.get('to'))



#print(doc_df1)

In [8]:

aspect_terms = pd.DataFrame(aspects_records)
aspect_terms

,from,id,polarity,term,to
0,8,3121,negative,staff,13
1,57,2777,positive,food,61
2,4,1634,positive,food,8
3,55,1634,positive,kitchen,62
4,141,1634,neutral,menu,145
5,17,2846,positive,food,21
6,51,2846,positive,perks,56
7,27,1458,positive,orrechiete with sausage and chicken,62
8,76,1458,positive,waiters,83
9,152,1458,neutral,meats,157


### Sentence Catergory Table

In [10]:
aspect_category_records = []
for node in etree.iter():
    record = {} 
    #print(node.attrib)
    #print(node.tag)
    
    
    if(node.tag == 'sentence'):
        id = node.attrib.get('id')
        #print("1:",id)
        
    if(node.tag == 'aspectCategory' ):
        
        #print("2",id)
        record['id'] = id
        record['category'] = node.attrib.get('category')
        record['polarity'] = node.attrib.get('polarity')
        
        aspect_category_records.append(record)

In [11]:
aspect_category = pd.DataFrame(aspect_category_records)
aspect_category

,category,id,polarity
0,service,3121,negative
1,food,2777,positive
2,anecdotes/miscellaneous,2777,negative
3,food,1634,positive
4,service,2534,positive
5,anecdotes/miscellaneous,583,positive
6,food,2846,positive
7,service,2846,positive
8,food,1571,negative
9,price,1571,negative


In [12]:
print("Number of data points", doc_df.shape[0])
print("Number of data points", doc_df.shape[1])
print("Number of data points", sentence_data.shape[0])
print("Number of data points", sentence_data.shape[1])

Number of data points 3044
Number of data points 1
Number of data points 3044
Number of data points 1


In [53]:
text_data = pd.concat([doc_df, sentence_data], axis=1)

In [54]:
text_data

,id,text
0,3121,But the staff was so horrible to us.
1,2777,"To be completely fair, the only redeeming fact..."
2,1634,"The food is uniformly exceptional, with a very..."
3,2534,Where Gabriela personaly greets you and recomm...
4,583,"For those that go once and don't enjoy it, all..."
5,2846,"Not only was the food outstanding, but the lit..."
6,1571,It is very overpriced and not very tasty.
7,1458,Our agreed favorite is the orrechiete with sau...
8,3161,The Bagels have an outstanding taste with a te...
9,2391,Nevertheless the food itself is pretty good.


In [82]:
text_data.columns

Index(['id', 'text'], dtype='object')

In [83]:
print("Number of features points", doc_df.shape[1])

Number of features points 1


In [84]:
doc_df.columns

Index(['id'], dtype='object')

### Basic Status of the data
1. Sentence Table (text_data)
2. Aspect Terms Data (aspect_terms)
3. Category Data (aspect_category)

In [55]:
#colums in text_data
text_data.columns

Index(['id', 'text'], dtype='object')

In [56]:
#describe text data
print(text_data['text'].describe())

count           3044
unique          3038
top       Good food.
freq               2
Name: text, dtype: object


In [88]:
#Aspect colums 
aspect_terms.columns

Index(['from', 'id', 'polarity', 'term', 'to'], dtype='object')

In [89]:
#Describe term in the aspects
print(aspect_terms['term'].describe())

count     3699
unique    1295
top       food
freq       358
Name: term, dtype: object


In [90]:
# Category columns
aspect_category.columns

Index(['category', 'id', 'polarity'], dtype='object')

In [91]:
# Describe category 
print(aspect_category['category'].describe())

count     3714
unique       5
top       food
freq      1233
Name: category, dtype: object


In [93]:
aspect_terms['term'].unique()

array(['staff', 'food', 'kitchen', ..., 'cheese sticks',
       'pot of boiling water', 'glass noodles'], dtype=object)

In [94]:
aspect_category['category'].unique()

array(['service', 'food', 'anecdotes/miscellaneous', 'price', 'ambience'], dtype=object)

In [105]:
# store thr sentence text data into the pickels/180k_text_data
text_data.to_pickle('pickels/180k_text_data')
aspect_terms.to_pickle('pickels/180k_aspect_data')
aspect_category.to_pickle('pickels/180k_category_data')

In [106]:
print("thie sedif dfdf dfdf".split())

['thie', 'sedif', 'dfdf', 'dfdf']


## 2. Text Preprocessing

In [336]:
# Remove the special charecter in the string 

string = "special character $# df_*d dfa chara spaces 88823434"
''.join(e for e in string if e.isalnum())


'specialcharacterdfddfacharaspaces88823434'

In [57]:
# I use the the  list of stop words that are downloaded from nltk lib.

stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)


def nlp_stopwordRemove(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            word = word.lower()
            # Remove stop words
            if not word in stop_words:
                string += word + " "
    text_data[column][index] = string

list of stop words: {'below', 'but', 'why', 'will', 'it', 'an', 'from', 'because', 'these', 'than', 'has', 'off', 'some', 'too', 'ma', 'haven', 'now', 'itself', 'aren', 'this', 'only', 'the', 'her', 'between', 'needn', 'their', 'couldn', 'she', 'again', 'me', 'so', 'through', 'about', 'into', 'or', 'wasn', 'are', 'don', 'll', 'be', 'herself', 'at', 'my', 'hers', 'up', 'which', 'him', 'his', 'them', 'down', 'in', 'shan', 'over', 'should', 'after', 'we', 'very', 'those', 'until', 're', 'theirs', 'himself', 'hadn', 'were', 'before', 'above', 'on', 'that', 't', 'can', 'what', 'while', 'mustn', 'a', 'yours', 'been', 'myself', 'ain', 'wouldn', 'to', 'against', 'when', 'd', 'ours', 'its', 'own', 'won', 'did', 'nor', 'being', 'other', 'such', 'each', 'both', 'm', 'isn', 'i', 'yourself', 'during', 'who', 'y', 'does', 'where', 'no', 'was', 'for', 'our', 'have', 'under', 'just', 'weren', 'there', 'doesn', 'shouldn', 'all', 've', 'he', 's', 'of', 'here', 'most', 'ourselves', 'they', 'had', 'out', 

In [58]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    nlp_stopwordRemove(row['text'], index, 'text')
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

0.5906178417772026 seconds


In [21]:
text_data

,id,text
0,3121,staff horrible us
1,2777,completely fair redeeming factor food average ...
2,1634,food uniformly exceptional capable kitchen pro...
3,2534,gabriela personaly greets recommends eat
4,583,go dont enjoy say dont get
5,2846,food outstanding little perks great
6,1571,overpriced tasty
7,1458,agreed favorite orrechiete sausage chicken usu...
8,3161,bagels outstanding taste terrific texture chew...
9,2391,nevertheless food pretty good


In [59]:
print(text_data['text'].describe())

count                    3044
unique                   3008
top       highly recommended 
freq                        4
Name: text, dtype: object


In [60]:
product_type_count = Counter(list(text_data['text']))
#product_type_count
product_type_count.most_common(40)

[('food great ', 4),
 ('highly recommended ', 4),
 ('love place ', 4),
 ('back ', 3),
 ('', 3),
 ('go ', 3),
 ('wonderful ', 3),
 ('disappointed ', 2),
 ('cant wait go back ', 2),
 ('check ', 2),
 ('dont go ', 2),
 ('enjoy ', 2),
 ('pleasantly surprised ', 2),
 ('finally ', 2),
 ('would highly recommend ', 2),
 ('great place ', 2),
 ('good food ', 2),
 ('horrible ', 2),
 ('delicious ', 2),
 ('really ', 2),
 ('amazing ', 2),
 ('definitely go back ', 2),
 ('food good ', 2),
 ('loved ', 2),
 ('unbelievable ', 2),
 ('service excellent food delicious ', 2),
 ('staff horrible us ', 1),
 ('completely fair redeeming factor food average couldnt make deficiencies teodora ',
  1),
 ('food uniformly exceptional capable kitchen proudly whip whatever feel like eating whether menu ',
  1),
 ('gabriela personaly greets recommends eat ', 1),
 ('go dont enjoy say dont get ', 1),
 ('food outstanding little perks great ', 1),
 ('overpriced tasty ', 1),
 ('agreed favorite orrechiete sausage chicken usually

In [61]:
data11 = ""
#for row in text_data.iterrows():
    #print(row['text'])
    #data11 += row['text']
sent = ""
for index, row in text_data.iterrows():
    sent += row['text']    
    
#print(sent.split(" "))

print(len(sent))

less_freq = {}


cou = Counter(list(sent.split(" ")))
print(cou)
for k,v in less_freq:
    if(v == 1):
        less_freq.add(k)
        print(k)

print(less_freq)
        

#print(len(cou))

less_freq = cou.most_common()[:-2435-1:-1]




#print (cou.values )

#for v in cou:


#print (data11.split())

'''
print(data11.split())
product_type_count = Counter(list(text_data['text']))
#product_type_count
product_type_count.most_common(40)
#.most_common(40)

#data1111 = data11.split()
'''





143497
Counter({'food': 453, 'place': 295, 'great': 261, 'good': 261, 'service': 247, 'restaurant': 189, 'go': 166, '': 165, 'like': 132, 'one': 123, 'best': 115, 'get': 110, 'would': 107, 'back': 92, 'dont': 88, 'ive': 85, 'time': 85, 'always': 85, 'delicious': 84, 'pizza': 83, 'menu': 82, 'try': 79, 'even': 77, 'dinner': 76, 'never': 75, 'wine': 74, 'nice': 74, 'well': 74, 'really': 72, 'staff': 71, 'excellent': 71, 'us': 70, 'table': 67, 'times': 66, 'people': 66, 'prices': 66, 'wait': 65, 'recommend': 64, 'little': 63, 'night': 63, 'friendly': 61, 'also': 61, 'better': 61, 'eat': 60, 'went': 59, 'thai': 59, 'experience': 58, 'restaurants': 57, 'sushi': 56, 'ever': 53, 'atmosphere': 52, 'new': 51, 'worth': 50, 'fresh': 50, 'dishes': 49, 'friends': 49, 'small': 49, 'price': 48, 'though': 48, 'could': 46, 'love': 46, 'special': 45, 'indian': 45, 'meal': 44, 'first': 44, 'nyc': 44, 'bar': 43, 'lunch': 43, 'chicken': 42, 'definitely': 42, 'got': 42, 'make': 41, 'city': 41, 'everything':

"\nprint(data11.split())\nproduct_type_count = Counter(list(text_data['text']))\n#product_type_count\nproduct_type_count.most_common(40)\n#.most_common(40)\n\n#data1111 = data11.split()\n"

### The seed words of each category
##### Unique Categories are 'service', 'food', 'anecdotes/miscellaneous', 'price', 'ambience'.

In [62]:
from nltk.corpus import wordnet
service_syns = wordnet.synsets("service")
print(service_syns[0].definition())
print(service_syns[0].examples())

#print (len(service_syns))
#i = len(service_syns)
for i in range(len(service_syns)):
    print(service_syns[i].lemmas()[0].name())

food_syns = wordnet.synsets("food")
print (food_syns)
for i in range(len(food_syns)):
    print(food_syns[i].lemmas()[0].name())

anecdotes_syns = wordnet.synsets("anecdotes")
print(anecdotes_syns)
anecdotes_syns = wordnet.synsets("miscellaneous")
print(anecdotes_syns)

pri = wordnet.synsets("price")
print(pri)

amb = wordnet.synsets("ambience")
print(amb)
#syns[0].name()
#syns[0].lemmas()[0].name()

work done by one person or group that benefits another
['budget separately for goods and services']
service
service
service
service
service
military_service
Service
avail
service
servicing
service
serve
service
overhaul
service
service
service
serve
[Synset('food.n.01'), Synset('food.n.02'), Synset('food.n.03')]
food
food
food
[Synset('anecdote.n.01')]
[Synset('assorted.s.01'), Synset('many-sided.s.02')]
[Synset('monetary_value.n.01'), Synset('price.n.02'), Synset('price.n.03'), Synset('price.n.04'), Synset('price.n.05'), Synset('price.n.06'), Synset('price.n.07'), Synset('price.v.01'), Synset('price.v.02')]
[Synset('atmosphere.n.01'), Synset('ambiance.n.02')]


In [240]:
import pandas as pd

df = pd.DataFrame({'TFD' : ['AA', 'SL', 'BB', 'D0', 'Dk', 'FF'],
                    'Snack' : ['1', '0', '1', '1', '0', '0'],
                    'Trans' : ['1', '1', '1', '0', '0', '1'],
                    'Dop' : ['1', '0', '1', '0', '1', '1']}).set_index('TFD')

print(df)

    Dop Snack Trans
TFD                
AA    1     1     1
SL    0     0     1
BB    1     1     1
D0    0     1     0
Dk    1     0     0
FF    1     0     1


In [241]:
df_asint = df.astype(int)
coocc = df_asint.T.dot(df_asint)
coocc

,Dop,Snack,Trans
Dop,4,2,3
Snack,2,3,2
Trans,3,2,4


In [242]:
np.fill_diagonal(coocc.values, 0)

In [243]:
coocc

,Dop,Snack,Trans
Dop,0,2,3
Snack,2,0,2
Trans,3,2,0


In [244]:
import scipy.sparse as sp
X = sp.csr_matrix(df.astype(int).values) # convert dataframe to sparse matrix
Xc = X.T * X # multiply sparse matrix # 
Xc.setdiag(0) # reset diagonal
print(Xc.todense()) # to print co-occurence matrix in dense format


[[0 2 3]
 [2 0 2]
 [3 2 0]]


In [355]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("texture"))

texture


##    Stemming

In [63]:
lemmatizer = WordNetLemmatizer()
def nlp_lemmatizing(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            word = word.lower()
            word = lemmatizer.lemmatize(word)
            string += word + " "
    text_data[column][index] = string


In [64]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    nlp_lemmatizing(row['text'], index, 'text')

# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")


0.7027635107770038 seconds


In [65]:
text_data

,id,text
0,3121,staff horrible u
1,2777,completely fair redeeming factor food average ...
2,1634,food uniformly exceptional capable kitchen pro...
3,2534,gabriela personaly greets recommends eat
4,583,go dont enjoy say dont get
5,2846,food outstanding little perk great
6,1571,overpriced tasty
7,1458,agreed favorite orrechiete sausage chicken usu...
8,3161,bagel outstanding taste terrific texture chewy...
9,2391,nevertheless food pretty good


In [66]:
product_type_count = Counter(list(text_data['text']))
product_type_count

Counter({'staff horrible u ': 1,
         'completely fair redeeming factor food average couldnt make deficiency teodora ': 1,
         'food uniformly exceptional capable kitchen proudly whip whatever feel like eating whether menu ': 1,
         'gabriela personaly greets recommends eat ': 1,
         'go dont enjoy say dont get ': 1,
         'food outstanding little perk great ': 1,
         'overpriced tasty ': 1,
         'agreed favorite orrechiete sausage chicken usually waiter kind enough split dish half get sample meat ': 1,
         'bagel outstanding taste terrific texture chewy yet gummy ': 1,
         'nevertheless food pretty good ': 1,
         'ive noticed megadeth mr scruff st germain traditional vietnamese song black sabbath jayz daft punk played ': 1,
         'mayonnaise forgot toast left ingredient ie cheese omelet hot temperature bacon cooked crumbled plate touched ': 1,
         'took half hour get check perfect since could sit drink talk ': 1,
         'design a

In [104]:
from itertools import dropwhile
sent = ""
for index, row in text_data.iterrows():
    sent += row['text']
#print (sent)
    
#print(sent.split(" "))

print(len(sent))

less_freq = {}

cou = Counter(list(sent.split(" ")))



cou1 = cou

#cou
#print(cou)

print (type(cou))

lis = list(cou1.items())
lis
#print(type(lis))
lis= set(lis)
#print (lis)
#print(type(lis))


for key, count in dropwhile(lambda key_count: key_count[1] > 1, cou.most_common()):
     del cou[key]
print (cou)



141388
<class 'collections.Counter'>
Counter({'food': 464, 'place': 330, 'great': 261, 'good': 261, 'service': 247, 'restaurant': 246, 'go': 170, 'time': 151, 'like': 134, 'one': 126, 'get': 119, 'best': 115, 'price': 114, 'would': 107, 'back': 92, 'pizza': 89, 'dont': 88, 'ive': 85, 'always': 85, 'delicious': 84, 'menu': 83, 'table': 83, 'wine': 82, 'dish': 81, 'try': 81, 'dinner': 78, 'even': 77, 'friend': 75, 'never': 75, 'nice': 74, 'well': 74, 'staff': 72, 'u': 72, 'really': 72, 'night': 71, 'excellent': 71, 'people': 66, 'wait': 65, 'recommend': 64, 'little': 63, 'special': 63, 'experience': 62, 'friendly': 61, 'also': 61, 'better': 61, 'make': 60, 'eat': 60, 'went': 59, 'thai': 59, 'sushi': 56, 'drink': 55, 'ever': 53, 'atmosphere': 52, 'new': 51, 'worth': 50, 'fresh': 50, 'small': 49, 'love': 49, 'though': 48, 'meal': 47, 'could': 46, 'indian': 45, 'bar': 44, 'first': 44, 'nyc': 44, 'order': 44, 'say': 43, 'come': 43, 'lunch': 43, 'chicken': 42, 'definitely': 42, 'got': 42, 'ci

##### Total Number of words remain is 4113
# of single word 2199 (4113 - 2199 = 1914)
# of single and two count words 2818 remaning(4113 - 2818 = 1295)


In [106]:
# Create a list whose frequency is greater than 2

#print(cou)

#cou = Counter(list(cou.split(" ")))
#print(cou)
#print (cou)

cou = Counter(list(sent.split(" ")))

low_fre_word = []




cnt = 0
for k, v in cou.items() :
    if(v < 4):
        cnt = cnt + 1
        low_fre_word.append(k)
        
        
print (cnt)

listt = Counter(low_fre_word)
listt = set(listt)

#print (listt)

#print (len(myList))
#print(low_fre_word)



lis2= set(low_fre_word)
#print (lis2)
#print(type(lis2))



print (lis - listt)





3075
{('rib', 2), ('ye', 1), ('shop', 2), ('polite', 1), ('ocasionally', 1), ('northern', 1), ('give', 28), ('sicilian', 1), ('grace', 2), ('ignore', 2), ('breast', 1), ('slighly', 1), ('spoke', 2), ('notorious', 2), ('ew', 2), ('pizzarias', 1), ('outrageously', 2), ('yucky', 1), ('threw', 2), ('5pm', 1), ('slurry', 1), ('ran', 2), ('wouldnt', 14), ('dinnerbroker', 1), ('understood', 1), ('aside', 4), ('waterbugs', 1), ('early', 7), ('basically', 2), ('40pp', 1), ('flowing', 1), ('stressed', 2), ('popular', 7), ('hand', 12), ('rarity', 1), ('thius', 1), ('sleeping', 1), ('whipped', 1), ('economical', 1), ('edamames', 1), ('donut', 1), ('zabars', 1), ('gelato', 3), ('rather', 13), ('moment', 6), ('stagnating', 1), ('wondering', 1), ('afternoon', 6), ('attentive', 34), ('frederick', 1), ('upscale', 1), ('overlyambitious', 1), ('unfriendly', 1), ('garden', 9), ('yuka', 2), ('horrific', 1), ('wintermelon', 1), ('personality', 2), ('resembled', 1), ('try', 81), ('ok', 21), ('ruby', 3), ('pr

In [48]:

# I use the the  list of stop words that have frequency 1.



def remove_lowFrequencyWord(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            # Remove stop words
            if not word in low_fre_word:
                string += word + " "
    text_data[column][index] = string




In [49]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    remove_lowFrequencyWord(row['text'], index, 'text')
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

1.2209775882497524 seconds


In [2]:
# After removing low frequency words text_data
text_data

NameError: name 'text_data' is not defined

### Co-occurance Matrix in Python

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
docs = ['this this this book',
        'this cat good',
        'cat good shit']
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model

X = count_model.fit_transform(docs)
#print(X)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
print(Xc)
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
print(Xc.todense()) # print out matrix in dense format


  (4, 0)	3
  (0, 0)	1
  (3, 1)	1
  (4, 1)	1
  (1, 1)	2
  (2, 1)	2
  (3, 2)	1
  (4, 2)	1
  (1, 2)	2
  (2, 2)	2
  (1, 3)	1
  (2, 3)	1
  (3, 3)	1
  (1, 4)	1
  (2, 4)	1
  (4, 4)	10
  (0, 4)	3
[[0 0 0 0 3]
 [0 0 2 1 1]
 [0 2 0 1 1]
 [0 1 1 0 0]
 [3 1 1 0 0]]


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
samples = ['awesome unicorns are awesome','batman forever and ever','I love batman forever']
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), vocabulary = {'awesome unicorns':0, 'batman forever':1}) 
co_occurrences = bigram_vectorizer.fit_transform(samples)
print ('Printing sparse matrix:', co_occurrences)
print ('Printing dense matrix (cols are vocabulary keys 0-> "awesome unicorns", 1-> "batman forever")', co_occurrences.todense())
sum_occ = np.sum(co_occurrences.todense(),axis=0)
print ('Sum of word-word occurrences:', sum_occ)
print ('Pretty printig of co_occurrences count:', zip(bigram_vectorizer.get_feature_names(),np.array(sum_occ)[0].tolist()))

Printing sparse matrix:   (0, 0)	1
  (1, 1)	1
  (2, 1)	1
Printing dense matrix (cols are vocabulary keys 0-> "awesome unicorns", 1-> "batman forever") [[1 0]
 [0 1]
 [0 1]]
Sum of word-word occurrences: [[1 2]]
Pretty printig of co_occurrences count: <zip object at 0x0000028A9C86E748>


In [15]:
# Check the word is preseint in the sentenec or not
mystring = "I love my country"
word = "country"
if word in mystring: 
    print ('success')

success


In [17]:
# For the position of the word in the sentence
str1In = "I like to code a few things and code a lot"
str2 = "code"
str1 = str1In.split()

indexCount = 0
for word in str1:
    if word == str2:
       print("Your word was found at index point",int(indexCount))
    
    indexCount += 1
    

Your word was found at index point 3
Your word was found at index point 8
